In [13]:
import requests
import json
import pandas as pd
from pandas import json_normalize
import config
from db_manager import DBManager
import os
from rest_api import Api
from datetime import date, datetime
from dateutil.relativedelta import relativedelta
from urllib.parse import quote_plus
from sqlalchemy import create_engine
import itertools
import logging
import pickle

In [14]:
logging.basicConfig(
    format='%(levelname)s-%(lineno)s-%(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    level='INFO'
)
logger=logging.getLogger(__name__)

#### Show all columns without truncating

Today's date

In [15]:
today = date.today()
logger.info(f'Current date: {today}')

INFO-__main__:2-Current date: 2023-10-25


Adding months to today's date. Near term travel dates 30 - 60 days ahead

In [16]:
departure_date_1_month = today + relativedelta(months=1)
departure_date_2_month = today + relativedelta(months=2)
return_date_1_month = departure_date_2_month
return_date_2_month = today + relativedelta(months=3)
logger.info(f'Departure_dates:{departure_date_1_month} and {departure_date_2_month} \nReturn dates:{return_date_1_month} and {return_date_2_month}')

INFO-__main__:5-Departure_dates:2023-11-25 and 2023-12-25 
Return dates:2023-12-25 and 2024-01-25


Get the nearby festival departure dates by start_date from the public.indian_holidays table

In [17]:
# credentials=config.pg_credentials
# db = DBManager(credentials=credentials)
# df_departure_arrival_dates = db.run_query(query_file_name='festival_dates')

In [5]:
#df_departure_arrival_dates 

Apply the combination of params to get data for all possible combinations of depart, arrival dates and depart arrival airports

Combination itertools usage. Im calculating the total combinations by multiplying the number of possibilities (unique values) for each variable<br />
Departure airport: 2 possibilities<br />
Arrival airport: 4 possibilities<br />
Departure date: 2 possibilities<br />
Arrival date: 1 possibilities<br />
Total combinations = 2 x 4 x 2 x 1 = 16 combinations

#### Amadeus flight offers price API to get flight price in real-time

--------------------------------------------------------

API test for amadeus

POST request to the Amadeus authorization server to get the access token

In [18]:
response = requests.post(url=config.url_token, headers=config.headers_token, data=config.data)

if response.status_code == 200:
  # API call succeeded
  token = response.json()['access_token'] 
  logger.info('Successful access token generation')
else:
  # API call failed
  print(response.text)

INFO-__main__:6-Successful access token generation


In [19]:
#Combinations of destination and origin airport codes
origins = config.params['originLocationCode']
destinations = config.params['destinationLocationCode']
departure_dates=[departure_date_1_month,departure_date_2_month]
# return_dates=[return_date_1_month,return_date_2_month]

### Loading the global variable

In [32]:
# Load the variable from the file
with open('global_variable.pkl', 'rb') as file:
    search_response = pickle.load(file)

Creating combinations of Origin and Destination to be passed as parameters to the api call

trial

In [34]:

# Define the request parameters
headers = {
    'client_id': config.client_id,
    'client_secret': config.client_secret,
    'Authorization': f'Bearer {token}'
}
url = config.url_price
search_response = search_response  # Your request data

# Make the POST request
try:
    response = requests.post(url, json=search_response, headers=headers)

    if response.status_code == 200:
        # Request was successful, process the response data
        data = response.json()
        print("Request was successful")
        print(data)
    elif response.status_code == 500:
        # Server Error - Handle and gather more information about the issue
        error_data = response.json()
        print("Server Error (500):")
        for error in error_data.get('errors', []):
            print(f"Status: {error.get('status')}")
            print(f"Code: {error.get('code')}")
            print(f"Title: {error.get('title')}")
            print(f"Detail: {error.get('detail')}")
        # You can log this information for debugging or report it as needed.
    else:
        # Handle other HTTP status codes here
        print(f"Request failed with status code: {response.status_code}")

except requests.exceptions.RequestException as e:
    # Handle connection errors, timeouts, and other exceptions
    print(f"Request Exception: {e}")

except Exception as e:
    # Handle other exceptions
    print(f"An unexpected error occurred: {e}")


Server Error (500):
Status: 500
Code: 38189
Title: Internal error
Detail: An internal error occurred, please contact your administrator


In [26]:
response

<Response [500]>